In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from glob import glob
import librosa
import warnings

warnings.filterwarnings("ignore")

from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, log_loss
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
class conv_bn_relu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(conv_bn_relu, self).__init__()
        
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, padding=kernel_size // 2)
        self.BN = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        
        
    def forward(self, x):
        x = self.conv(x)
        x = self.BN(x)
        x = self.relu(x)
        return x
    

class Network(nn.Module):
    def __init__(self, N):
        super(Network, self).__init__()
        self.N = N
        self.AveragePooling = nn.AvgPool2d(2)
        self.MaxPooling = nn.MaxPool2d(2)
        
        
        self.input_conv = conv_bn_relu(in_channels=64, out_channels=self.N, kernel_size=3)
        
        self.block1 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )
        
        self.block2 = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*4, kernel_size=3),
            conv_bn_relu(in_channels=self.N*4, out_channels=self.N*2, kernel_size=3),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*1, kernel_size=3),
        )
        
        self.pool_block = nn.Sequential(
            conv_bn_relu(in_channels=self.N*1, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AvgPool2d(2),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AvgPool2d(2),
            conv_bn_relu(in_channels=self.N*2, out_channels=self.N*2, kernel_size=3),
            nn.Dropout(0.15),
            nn.AdaptiveAvgPool2d(1)
        )
        
        self.output = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=self.N*2, out_features=6),
        )
        
    def forward(self, x, out=''):
        x = self.input_conv(x)
        x = self.MaxPooling(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.pool_block(x)
        x=self.output(x)
        
        if out=='sigmoid':
            x = F.sigmoid(x)
        return x

In [12]:
del train_x
del train_y
del test_x
torch.cuda.empty_cache()

In [3]:
train_x = torch.tensor(np.load("./data/train_x_shf.npy", allow_pickle = True), dtype=torch.float32)
train_y = torch.tensor(np.load("./data/train_y_shf.npy", allow_pickle = True), dtype=torch.float32)
test_x = torch.tensor(np.load("./data/test_x_shf.npy", allow_pickle = True), dtype=torch.float32)


(torch.Size([25520, 1, 64, 501]),
 torch.Size([25520]),
 torch.Size([6100, 1, 64, 501]))

In [4]:
skf = StratifiedKFold(n_splits=5, random_state=61, shuffle=True)
for train_idx, val_idx in skf.split(train_x, train_y):
    train_ds = TensorDataset(train_x[train_idx], train_y[train_idx])
    train_loader = DataLoader(dataset=train_ds, batch_size=32, num_workers=2)
    val_ds = TensorDataset(train_x[val_idx], train_y[val_idx])
    val_loader = DataLoader(dataset=val_ds, batch_size=32, num_workers=2)
    
    model = Network(16).to(device)
    now = time.localtime()
    model_name = f'{now.tm_mon:02d}{now.tm_mday:02d} {now.tm_hour:02d}{now.tm_min:02d}'
    
    opt = torch.optim.Adam(model.parameters(), lr =1e-3)
    loss_fn = nn.CrossEntropyLoss()
    best = 9999
    
    epochs = 30
    for epoch in range(epochs):
        start = time.time()
        model.train()
        train_loss = 0
        pred_list, true_list = [], []
        for _, (x,y) in enumerate(train_loader):
            x = x.to(device)
            y = y.type(torch.LongTensor).to(device)
            
            opt.zero_grad()
            pred = model(x)
            loss = loss_fn(pred, y)
            loss.backward()
            opt.step()
            
            train_loss += loss.item()
            pred_list += F.softmax(pred).argmax(1).detach().cpu().numpy().tolist()
            true_list += y.detach().cpu().numpy().tolist()
            
        train_acc = accuracy_score(true_list, pred_list)
        
        with torch.no_grad():
            model.eval()
            val_loss = 0
            pred_list, true_list = [], []
            for _, (x, y) in enumerate(val_loader):
                x = x.to(device)
                y = y.type(torch.LongTensor).to(device)
                
                pred = model(x)
                loss = loss_fn(pred, y)
                
                val_loss += loss.item()
                pred_list += F.softmax(pred).argmax(1).detach().cpu().numpy().tolist()
                true_list += y.detach().cpu().numpy().tolist()
                
        val_acc = accuracy_score(true_list, pred_list)
        
        if val_loss/len(val_loader) < best:
            best = val_loss/len(val_loader)
            torch.save({'model':model.state_dict()}, f'./model/result/{model_name}.pt')
        
        print(f'===================== Epoch : {epoch+1}/{epochs}    time : {time.time()-start:.0f}s =====================')
        print(f'TRAIN -> loss : {train_loss/len(train_loader):.5f}     accuracy : {train_acc:.5f}')
        print(f'VALID -> loss : {val_loss/len(val_loader):.5f}     accuracy : {val_acc:.5f}    best : {best:.5f}\n\n')

===================== Epoch : 1/30    time : 88s =====================
TRAIN -> loss : 0.85692     accuracy : 0.77802
VALID -> loss : 2.80049     accuracy : 0.39185    best : 2.80049


===================== Epoch : 2/30    time : 80s =====================
TRAIN -> loss : 0.77682     accuracy : 0.76538
VALID -> loss : 2.85877     accuracy : 0.39185    best : 2.80049


===================== Epoch : 3/30    time : 80s =====================
TRAIN -> loss : 0.65329     accuracy : 0.79002
VALID -> loss : 3.28677     accuracy : 0.39185    best : 2.80049


===================== Epoch : 4/30    time : 80s =====================
TRAIN -> loss : 0.73447     accuracy : 0.76220
VALID -> loss : 3.23649     accuracy : 0.39185    best : 2.80049




KeyboardInterrupt: 